#### Dataset Correction
- To be used when reformatting datasets to correct minor errors that may impact fine-tuning immensely
- Current Functionality : Generates Train Test Splits from a main dataset, there are no more plans on expanding the dataset further

In [ ]:

import pandas as pd

## CSV Properties
csv = "cycle3_full.csv"
csv_list = ["cycle3_1000_train.csv",
        "cycle3_1000_test.csv",
        "cycle3_2000_train.csv",
        "cycle3_2000_test.csv",
        "cycle3_3000_train.csv",
        "cycle3_3000_test.csv",
        "cycle3_train.csv",
        "cycle3_test.csv",
        "cycle3_1000_trainformatted.csv",
        "cycle3_1000_testformatted.csv",
        "cycle3_2000_trainformatted.csv",
        "cycle3_2000_testformatted.csv",
        "cycle3_3000_trainformatted.csv",
        "cycle3_3000_testformatted.csv",
        "cycle3_trainformatted.csv",
        "cycle3_testformatted.csv",
        ]
sep = "," # \t
random_state = 4000

## Input Replacement
input_replacement = """You are an expert in software quality assurance specializing in usability testing.

Consider the following usability aspects:
Accessibility (keyboard navigation, screen reader support, etc.)
Responsiveness (behavior across different screen sizes, etc.)
Feedback (hover effects, click responses, error messages, etc.)
Interactivity (expected behavior when clicked, typed into, or focused, etc.)
User experience (clarity of labels, ease of use, etc.)

Avoid descriptions like "The user" or "The test participant" and should instead focus on direct, executable actions.

The functional usability test case has the following aspects, separated by a `~`:
- **Objective**: Clearly state the objective of the test.
- **Preconditions**: List any preconditions that need to be met.
- **Test Steps**: Provide a step-by-step guide for the test. Each step should be a direct command, not a description.
- **Expected Output**: Describe the expected output.

Output format example, start with the objective directly and avoid saying "here is the test case" or the name of the testcase and end the output after writing the expected output:
Objective: [Describe the objective] 
~ Preconditions: [List preconditions, use dashes] 
~ Test Steps: [Step-by-step guide] 
~ Expected Output: [Describe the expected output, use dashes]"""
## Instruction Replacement
instruction_text = "Generate test case for the following UI element:"
instruction_text_replacement = "Generate a functional usability test case for the following UI element:"

# Correction / Reformatting Procedure
def correction_dataset(df, correction_procedure : str = "A", url : str = ""):
    # [DEPRECATED] Correction Procedure A : Convert Instruction Column Contents to Instruction+Input, Clear Input Contents
    if (correction_procedure == "A"):
        i = 0
        while i<len(df):
            df['instruction'][i] = df['instruction'][i].replace(df['instruction'][i], df['instruction'][i] + " " + df['input'][i])
            df['input'][i] = df['input'][i].replace(df['input'][i], "")
            i+=1
    # [DEPRECATED] Correction Procedure B : Append URL
    if (correction_procedure == "B"):
        i = 0
        while i<len(df):
            df['instruction'][i] = df['instruction'][i].replace(df['instruction'][i], df['instruction'][i] + " " + url)
            i+=1
    # Correction Procedure C : Input Replacement
    if (correction_procedure == "C"):
        i = 0
        while i<len(df):
            df['input'][i] = input_replacement
            i+=1
    # Correction Procedure D : Instruction Replacement
    if (correction_procedure == "D"):
        i = 0
        while i<len(df):
            df['instruction'][i] = df['instruction'][i].replace(instruction_text, "Generate a functional usability test case for the following UI element:")
    
    return df

# Returns a new df with different rows
def dataset_sampler(df, sample_size, limit = 3000):
    if sample_size > limit:
        sample_df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    else:
        sample_df = df.sample(sample_size, random_state=random_state).reset_index(drop=True)
    return sample_df

# Returns a split dataset
def dataset_splitter(df, size, first_split_percentage, limit = 3000):

    if size > limit:
        first_split_size = int(first_split_percentage * len(df))
    else:
        first_split_size = int(first_split_percentage * size)

    # Splits
    first_split = df.iloc[:first_split_size, :].reset_index(drop=True)
    second_split = df.iloc[first_split_size:, :].reset_index(drop=True)

    # Return Splits
    splits = [first_split, second_split]
    return splits

##### Main Program

In [ ]:
## RUN SETTINGS
RUN_SETTINGS = 0

## RUN -1 : [DEPRECATED] Correct Collected Dataset Based on Batch Settings
if RUN_SETTINGS == -1:
    csvs = ["cycle3_test.csv", "cycle3_train.csv"]
    urls = ["", ""]
    batch = True
    setting = "C"

    if (batch == True):
        i = 0
        while i<len(csvs):
            df = pd.read_csv(csvs[i], sep=sep)
            print(df)
            df = correction_dataset(df, correction_procedure=setting, url=urls[i])
            print(df)
            df.to_csv(f"{csvs[i].replace(".csv", "")}formatted.csv", sep=sep, encoding='utf-8', index=False, header=True)
            i += 1
    else:
        df = pd.read_csv(csvs[0], sep=sep)
        print(df)
        df = correction_dataset(df, correction_procedure=setting, url=urls[0])
        print(df)
        df.to_csv(f"{csvs[0].replace(".csv", "")}formatted.csv", sep=sep, encoding='utf-8', index=False, header=True)

## RUN 0: From a full dataset, sample to several split datasets with different sizes
if RUN_SETTINGS == 0:
    limit = 3000
    df = pd.read_csv(csv, sep=sep)
    sample_list = [1000, 2000, 3000, 4000]
    first_split = .8
    split_names = ["train", "test"]
    for sample_size in sample_list:
        # Get Main CSV dataset
        dataframe = dataset_sampler(df, sample_size)
        splits = dataset_splitter(dataframe, sample_size, .8)
        i = 0
        for i in range(len(splits)):
            if sample_size > limit:
                splits[i].to_csv(f"{csv.replace("_full.csv", "")}_{split_names[i]}.csv", sep=sep, encoding='utf-8', index=False, header=True)
                formatted_split = correction_dataset(splits[i], "C")
                formatted_split.to_csv(f"{csv.replace("_full.csv", "")}_{split_names[i]}formatted.csv", sep=sep, encoding='utf-8', index=False, header=True)
            else:
                splits[i].to_csv(f"{csv.replace("_full.csv", "")}_{sample_size}_{split_names[i]}.csv", sep=sep, encoding='utf-8', index=False, header=True)
                formatted_split = correction_dataset(splits[i], "C")
                formatted_split.to_csv(f"{csv.replace("_full.csv", "")}_{sample_size}_{split_names[i]}formatted.csv", sep=sep, encoding='utf-8', index=False, header=True)

## RUN 1: From a batch of datasets, correct input and instruction contents
if RUN_SETTINGS == 1:
    for csv_item in csv_list:
        df = pd.read_csv(csv_item, sep=sep)
        # Input Replacement
        df = correction_dataset(df, "C")
        # Output Replacement
        df = correction_dataset(df, "D")
        # Replace Currently Existing CSV
        df.to_csv(csv_item, sep=sep, encoding='utf-8', index=False, header=True)


##### Input Replacements

In [ ]:
##### Last Used on 03-14-2025
"""You are an expert in software quality assurance specializing in usability testing.

Consider the following usability aspects:
Accessibility (keyboard navigation, screen reader support, etc.)
Responsiveness (behavior across different screen sizes, etc.)
Feedback (hover effects, click responses, error messages, etc.)
Interactivity (expected behavior when clicked, typed into, or focused, etc.)
User experience (clarity of labels, ease of use, etc.)

Avoid descriptions like "The user" or "The test participant" and should instead focus on direct, executable actions.

Generate a functional usability test case with the following aspects, separated by a `~`:
- **Objective**: Clearly state the objective of the test.
- **Preconditions**: List any preconditions that need to be met.
- **Test Steps**: Provide a step-by-step guide for the test. Each step should be a direct command, not a description.
- **Expected Output**: Describe the expected output.

Output format example, start with the objective directly and avoid saying "here is the test case" or the name of the testcase and end the output after writing the expected output:
Objective: [Describe the objective] 
~ Preconditions: [List preconditions, use dashes] 
~ Test Steps: [Step-by-step guide] 
~ Expected Output: [Describe the expected output, use dashes]"""

In [ ]:
##### Last Used During Cycle 2
"""You are an expert in software quality assurance specializing in usability testing. Given a UI element, generate a set of functional usability test cases with detailed test scenarios.

    The UI element could be a button, link, text, or input field (some elements have a link attached to them). Consider the attribute attached to the UI element as the attribute acts as a hint about the primary focus of the test case. Also consider the following usability aspects:

    Accessibility (keyboard navigation, screen reader support, etc.)
    Responsiveness (behavior across different screen sizes, etc.)
    Feedback (hover effects, click responses, error messages, etc.)
    Interactivity (expected behavior when clicked, typed into, or focused, etc.)
    User experience (clarity of labels, ease of use, etc.)

    Only output the following, separated by a ~:

    Objective (What the test aims to verify)
    Preconditions (Any setup or conditions required before testing)
    Test Steps (Step-by-step actions to perform the test)
    Expected Result (Expected outcome if the UI element works correctly)

    Your output should only be in the following structured format but do not include the format in the output: "Objective"~"Preconditions"~"Test Steps"~"Expected Result"
    DO NOT output anything else but one test case in the said format.

    Here is the UI Element: {ui_element}
    The UI Element is from the following website URL : {url}"""

##### Instruction Replacements

In [ ]:
##### Original: A Typo Error In Dataset Generation
instruction_text = "Generate test case for the following UI element:"
# Above text should be generate A test case, and adding currently used context/input, we change it to:
instruction_text_replacement = "Generate a functional usability test case for the following UI element:"